# Simulation process notebook

## Preparation

In [ ]:
import math
import json
import pickle
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error # RMSE for forecast error

year = '2014-2016'
year = '2017' # Comment this line for 2014-2016 predictions
file_path = '../data/MergedData' + year + '.xlsx'
df = pd.read_excel(file_path)
df['Date'] = pd.to_datetime(df.Date, format='%Y%m%d')
df.rename(columns={'# items demanded':'demand',
                  'Avg temp in 0.1oC': 'temperature',
                  'Rainfall in 24h in 0.1mm':'precipitation',
                  }, inplace=True)

## Approach 1: Running the simulation with moving average on the train (2014-2016) or test (2017) set

In [ ]:
z = 2 # Regular safety factor
# z = 2.5 # For found optimum value of z; uncomment this entire line
m = 5  # Shelf life
w = 100  # Waste cost (surplus cost)
u = 500  # Lost sales cost (shortage cost)
Twarm = 14  # Number of warmup days

D_obs = df['demand'].values  # Assign demand vector (from csv) to D_obs: this is observed demand
D_fcst = []  # Assign forecast demand vector (from csv) to D_fcst

def fcst_error(demand_true, demand_pred):
    # Calculate fcst error based on true and predicted demand so far
    return math.sqrt(mean_squared_error(demand_true, demand_pred))

def S_rmse(d, demand_true, demand_pred):
    # Calculate order-up-to-level S based on RMSE fcst error
    return round(d + 2*fcst_error(demand_true, demand_pred))

def S_rmse_alt(d, squared_errors, z):
    rmse = math.sqrt(np.mean(squared_errors))
    return round(d + z*rmse)

I = [0, 0, 0, 0, 0]  # Initialise inventory array of only 0's
TotWaste = 0  # Initialise waste
TotOrdered = 0  # Initialise total ordered
TotDemand = 0 # Initialise total demand
NShort = 0  # Initialise shortage
TC = 0 # Initialise total cost variable
fillrateDay = [] # Initialise fill rate per day array
squared_errors = [] # Initialise empty array for squared errors per day
today_short = 0 # Initialise zero for today shortage

for t in range(Twarm, len(D_obs)-1):
    mu_t = (D_obs[t - 7] + D_obs[t - 14]) / 2  # Moving average t
    mu_t1 = (D_obs[t - 6] + D_obs[t - 13]) / 2  # Moving average t+1
    mu_lr = mu_t + mu_t1 # Forecast demand for today and tomorrow
    D_fcst.append(mu_lr) # Append to array of fcst t and t+1
    
#   Calculate squared error based on d_{t} and d_{t+1} and append to list
#   of squared errors
    demand_t_and_t_plus_1_true = D_obs[t] + D_obs[t+1]
    squared_error = (demand_t_and_t_plus_1_true - mu_lr)**2
    squared_errors.append(squared_error)

    # Determine s_t based on RMSE-like fcst error
    s_t = S_rmse_alt(mu_lr, squared_errors, z)
#     s_t = S_rmse(mu_t, D_obs[Twarm:t+1], D_fcst)
    q = max(0, (s_t - sum(I)))  # Determine order quantity q for next day
    d_observed = D_obs[t]  # Read value from array of demand D_obs
    TotDemand += d_observed # Add current demand to total to obtain sum after
    # warm-up period

    # Determine fill rate for the day
    if d_observed == 0:
        fillrateDay.append(1)  # Set fill rate to 1 in case demand is 0

    elif d_observed >= sum(I): # If demand is larger than total inventory
        fillrateDay.append(sum(I) / d_observed) # Append fill rate to fill rate list
        today_short = d_observed - sum(I)# We have shortage: lost sales
        NShort += today_short # Add shortage to total shortage
        I = [0, 0, 0, 0, 0] # Reset I to only zeros, because we have sold out

    else:
        fillrateDay.append(1)
        for i in range(m):  # Inventory is taken full FIFO (old -> new)

            # Procedure for handling demand; waterfall like approach
            if I[i] > d_observed: # If we have more inventory of exp date i than demand
                I[i] = I[i] - d_observed # Remaining inventory is lessened
                d_observed = 0 # D observed is fulfilled, so set to 0 and break out of loop 
                break
            else: # If d_obs >= I[i]; handle demand and continue looping
                d_observed = d_observed - I[i]
                I[i] = 0

    # Determine stats at day's end
    today_waste = I[0]  # Count expired products as waste
    TotWaste += today_waste
    TotOrdered += q  # Add ordered quantity to total ordered quantity
    TC += (u * today_short) + (w * today_waste)
    today_short = 0 # Reset today_short to 0 again as it's not always assigned

    # Shift the day window and add newly ordered goods to inventory
    I = I[1:]
    I.append(q)

result_stats = {
    "Waste": round(TotWaste/TotOrdered, 3),
    "Short": round(NShort/TotDemand, 3),
    "Avg cost (day)": round(TC / (len(D_obs) - Twarm)),
}

print(json.dumps(result_stats, indent=3))

## Approach 2: ML demand forecast

Additional steps:
- Load the relevant code from different modules: `DemandForecast` and `load_trained_xgb_regressor`
- Print out the results

### Load required set-up code

In [ ]:
# Example execution of main process
from demand_forecast import DemandForecast
from load_model import load_trained_xgb_regressor

# 1) Load the trained xgb model
xgb_tuned = load_trained_xgb_regressor()

# 2) Instantiate the class and predict on new data
fcst = DemandForecast(file_path = '../data/MergedData' + year + '.xlsx',
                     prediction_model=xgb_tuned)
fcst.prepare_data() # Prepares the input data (adds weekdayand lagged vars)
fcst.predict_demand() # Predicts demand and concats to dataframe

### Run the simulation based on the XGB-predicted demand (for 2014-2016 or 2017)

In [ ]:
z = 2 # Regular safety factor
# z = 2.5 # For found optimum value of z; uncomment this entire line
m = 5  # Shelf life
w = 100  # Waste cost (surplus cost)
u = 500  # Lost sales cost (shortage cost)
Twarm = 14  # Number of warmup days

n_obs = len(fcst._data)  # Assign demand vector (from csv) to D_obs: this is observed demand

def S_rmse_alt(d, squared_errors, z):
    rmse = math.sqrt(np.mean(squared_errors))
    return round(d + z*rmse)

I = [0, 0, 0, 0, 0]  # Initialise inventory array of only 0's
TotWaste = 0  # Initialise waste
TotOrdered = 0  # Initialise total ordered
TotDemand = 0 # Initialise total demand
NShort = 0  # Initialise shortage
TC = 0 # Initialise total cost variable
fillrateDay = [] # Initialise fill rate per day array
squared_errors = [] # Initialise empty array for squared errors per day

for t in range(Twarm, n_obs-1):
    mu_t, mu_t1 = fcst.get_prediction_for_day(t) #Returns tuple (t, t+1) pred
    mu_lr = mu_t + mu_t1 # Forecast demand for today and tomorrow
    
#   Calculate squared error based on d_{t} and d_{t+1} and append to list
#   of squared errors
    d_obs_t, d_obs_t1 = fcst.get_observed_demand_for_day(t)
    squared_error = ((d_obs_t + d_obs_t1) - mu_lr)**2
    squared_errors.append(squared_error)

    # Determine s_t based on RMSE-like fcst error
    s_t = S_rmse_alt(mu_lr, squared_errors, z)
    q = max(0, (s_t - sum(I)))  # Determine order quantity q for next day
    d_observed = D_obs[t]  # Read value from array of demand D_obs
    TotDemand += d_observed # Add current demand to total to obtain sum after
    # warm-up period

    # Determine fill rate for the day
    if d_observed == 0:
        fillrateDay.append(1)  # Set fill rate to 1 in case demand is 0

    elif d_observed >= sum(I): # If demand is larger than total inventory
        fillrateDay.append(sum(I) / d_observed) # Append fill rate to fill rate list
        today_short = d_observed - sum(I)# We have shortage: lost sales
        NShort += today_short # Add shortage to total shortage
        I = [0, 0, 0, 0, 0] # Reset I to only zeros, because we have sold out

    else:
        fillrateDay.append(1)
        for i in range(m):  # Inventory is taken full FIFO (old -> new)

            # Procedure for handling demand; waterfall like approach
            if I[i] > d_observed: # If we have more inventory of exp date i than demand
                I[i] = I[i] - d_observed # Remaining inventory is lessened
                d_observed = 0 # D observed is fulfilled, so set to 0 and break out of loop 
                break
            else: # If d_obs >= I[i]; handle demand and continue looping
                d_observed = d_observed - I[i]
                I[i] = 0

    # Determine stats at day's end
    today_waste = I[0]  # Count expired products as waste
    TotWaste += today_waste
    TotOrdered += q  # Add ordered quantity to total ordered quantity
    TC += (u * today_short) + (w * today_waste)
    today_short = 0 # Reset today_short to 0 again as it's not always assigned

    # Shift the day window and add newly ordered goods to inventory
    I = I[1:]
    I.append(q)

result_stats = {
    "Waste": round(TotWaste/TotOrdered, 3),
    "Short": round(NShort/TotDemand, 3),
    "Avg cost (day)": round(TC / (n_obs - Twarm)),
}

print(json.dumps(result_stats, indent=3))

## Prescriptive analytics

- For safety factor _z_, we seek the optimum value
- Hence we perform an exhaustive search on the close neighbourhood of _z_'s original value to determine its optimum with respect to costs
- We take costs as the cost function, because it combines both waste and shortage, which is exactly what we want from an objective/cost function

In [ ]:
m = 5  # Shelf life
w = 100  # Waste cost (surplus cost)
u = 500  # Lost sales cost (shortage cost)
Twarm = 14  # Number of warmup days
result_stats = dict()
n_obs = len(fcst._data)  # Assign demand vector (from csv) to D_obs: this is observed demand

def S_rmse_alt(d, squared_errors, z):
    rmse = math.sqrt(np.mean(squared_errors))
    return round(d + z*rmse)

# Test for multiple values of z
for z in [i for i in np.arange(0.5, 6, 0.5)]:

    I = [0, 0, 0, 0, 0]  # Initialise inventory array of only 0's
    TotWaste = 0  # Initialise waste
    TotOrdered = 0  # Initialise total ordered
    TotDemand = 0 # Initialise total demand
    NShort = 0  # Initialise shortage
    TC = 0 # Initialise total cost variable
    fillrateDay = [] # Initialise fill rate per day array
    squared_errors = [] # Initialise empty array for squared errors per day
    
    for t in range(Twarm, n_obs-1):
        mu_t, mu_t1 = fcst.get_prediction_for_day(t) #Returns tuple (t, t+1) pred
        mu_lr = mu_t + mu_t1 # Forecast demand for today and tomorrow

    #   Calculate squared error based on d_{t} and d_{t+1} and append to list
    #   of squared errors
        d_obs_t, d_obs_t1 = fcst.get_observed_demand_for_day(t)
        squared_error = ((d_obs_t + d_obs_t1) - mu_lr)**2
        squared_errors.append(squared_error)

        # Determine s_t based on RMSE-like fcst error
        s_t = S_rmse_alt(mu_lr, squared_errors, z)
        q = max(0, (s_t - sum(I)))  # Determine order quantity q for next day
        d_observed = D_obs[t]  # Read value from array of demand D_obs
        TotDemand += d_observed # Add current demand to total to obtain sum after
        # warm-up period

        # Determine fill rate for the day
        if d_observed == 0:
            fillrateDay.append(1)  # Set fill rate to 1 in case demand is 0

        elif d_observed >= sum(I): # If demand is larger than total inventory
            fillrateDay.append(sum(I) / d_observed) # Append fill rate to fill rate list
            today_short = d_observed - sum(I)# We have shortage: lost sales
            NShort += today_short # Add shortage to total shortage
            I = [0, 0, 0, 0, 0] # Reset I to only zeros, because we have sold out

        else:
            fillrateDay.append(1)
            for i in range(m):  # Inventory is taken full FIFO (old -> new)

                # Procedure for handling demand; waterfall like approach
                if I[i] > d_observed: # If we have more inventory of exp date i than demand
                    I[i] = I[i] - d_observed # Remaining inventory is lessened
                    d_observed = 0 # D observed is fulfilled, so set to 0 and break out of loop 
                    break
                else: # If d_obs >= I[i]; handle demand and continue looping
                    d_observed = d_observed - I[i]
                    I[i] = 0

        # Determine stats at day's end
        today_waste = I[0]  # Count expired products as waste
        TotWaste += today_waste
        TotOrdered += q  # Add ordered quantity to total ordered quantity
        TC += (u * today_short) + (w * today_waste)
        today_short = 0 # Reset today_short to 0 again as it's not always assigned

        # Shift the day window and add newly ordered goods to inventory
        I = I[1:]
        I.append(q)

    result_stats["Avg cost (day) " + str(z)] = \
                    round(TC / (n_obs - Twarm))

print(json.dumps(result_stats, indent=3))

### Run the simulation with the moving average model on the validation set of 2018

In [ ]:
file_path_to_validation_set = "../data/MergedData2018.xlsx"
df = pd.read_excel(file_path_to_validation_set)
df['Date'] = pd.to_datetime(df.Date, format='%Y%m%d')
df.rename(columns={'# items demanded':'demand',
                  'Avg temp in 0.1oC': 'temperature',
                  'Rainfall in 24h in 0.1mm':'precipitation',
                  }, inplace=True)

z = 2 # Regular safety factor
# z = 2.5 # For found optimum value of z; uncomment this entire line
m = 5  # Shelf life
w = 100  # Waste cost (surplus cost)
u = 500  # Lost sales cost (shortage cost)
Twarm = 14  # Number of warmup days

D_obs = df['demand'].values  # Assign demand vector (from csv) to D_obs: this is observed demand
D_fcst = []  # Assign forecast demand vector (from csv) to D_fcst

def fcst_error(demand_true, demand_pred):
    # Calculate fcst error based on true and predicted demand so far
    return math.sqrt(mean_squared_error(demand_true, demand_pred))

def S_rmse(d, demand_true, demand_pred):
    # Calculate order-up-to-level S based on RMSE fcst error
    return round(d + 2*fcst_error(demand_true, demand_pred))

def S_rmse_alt(d, squared_errors, z):
    rmse = math.sqrt(np.mean(squared_errors))
    return round(d + z*rmse)

I = [0, 0, 0, 0, 0]  # Initialise inventory array of only 0's
TotWaste = 0  # Initialise waste
TotOrdered = 0  # Initialise total ordered
TotDemand = 0 # Initialise total demand
NShort = 0  # Initialise shortage
TC = 0 # Initialise total cost variable
fillrateDay = [] # Initialise fill rate per day array
squared_errors = [] # Initialise empty array for squared errors per day
today_short = 0 # Initialise zero for today shortage

for t in range(Twarm, len(D_obs)-1):
    mu_t = (D_obs[t - 7] + D_obs[t - 14]) / 2  # Moving average t
    mu_t1 = (D_obs[t - 6] + D_obs[t - 13]) / 2  # Moving average t+1
    mu_lr = mu_t + mu_t1 # Forecast demand for today and tomorrow
    D_fcst.append(mu_lr) # Append to array of fcst t and t+1
    
#   Calculate squared error based on d_{t} and d_{t+1} and append to list
#   of squared errors
    demand_t_and_t_plus_1_true = D_obs[t] + D_obs[t+1]
    squared_error = (demand_t_and_t_plus_1_true - mu_lr)**2
    squared_errors.append(squared_error)

    # Determine s_t based on RMSE-like fcst error
    s_t = S_rmse_alt(mu_lr, squared_errors, z)
#     s_t = S_rmse(mu_t, D_obs[Twarm:t+1], D_fcst)
    q = max(0, (s_t - sum(I)))  # Determine order quantity q for next day
    d_observed = D_obs[t]  # Read value from array of demand D_obs
    TotDemand += d_observed # Add current demand to total to obtain sum after
    # warm-up period

    # Determine fill rate for the day
    if d_observed == 0:
        fillrateDay.append(1)  # Set fill rate to 1 in case demand is 0

    elif d_observed >= sum(I): # If demand is larger than total inventory
        fillrateDay.append(sum(I) / d_observed) # Append fill rate to fill rate list
        today_short = d_observed - sum(I)# We have shortage: lost sales
        NShort += today_short # Add shortage to total shortage
        I = [0, 0, 0, 0, 0] # Reset I to only zeros, because we have sold out

    else:
        fillrateDay.append(1)
        for i in range(m):  # Inventory is taken full FIFO (old -> new)

            # Procedure for handling demand; waterfall like approach
            if I[i] > d_observed: # If we have more inventory of exp date i than demand
                I[i] = I[i] - d_observed # Remaining inventory is lessened
                d_observed = 0 # D observed is fulfilled, so set to 0 and break out of loop 
                break
            else: # If d_obs >= I[i]; handle demand and continue looping
                d_observed = d_observed - I[i]
                I[i] = 0

    # Determine stats at day's end
    today_waste = I[0]  # Count expired products as waste
    TotWaste += today_waste
    TotOrdered += q  # Add ordered quantity to total ordered quantity
    TC += (u * today_short) + (w * today_waste)
    today_short = 0 # Reset today_short to 0 again as it's not always assigned

    # Shift the day window and add newly ordered goods to inventory
    I = I[1:]
    I.append(q)

result_stats = {
    "Waste": round(TotWaste/TotOrdered, 3),
    "Short": round(NShort/TotDemand, 3),
    "Avg cost (day)": round(TC / (len(D_obs) - Twarm)),
}

print(json.dumps(result_stats, indent=3))

### Run the simulation with the XGB model on the validation set of 2018

In [ ]:
# Example execution of main process
from demand_forecast import DemandForecast
from load_model import load_trained_xgb_regressor

# 1) Load the trained xgb model
xgb_tuned = load_trained_xgb_regressor()

# 2) Instantiate the class and predict on new data
file_path_to_validation_set = "../data/MergedData2018.xlsx"

fcst_2018 = DemandForecast(file_path = file_path_to_validation_set,
                     prediction_model = xgb_tuned)

fcst_2018.prepare_data() # Prepares the input data (adds weekdays and lagged vars)
fcst_2018.predict_demand() # Predicts demand and concats to dataframe

z = 2 # Regular safety factor
# z = 2.5 # For found optimum value of z; uncomment this entire line
m = 5  # Shelf life
w = 100  # Waste cost (surplus cost)
u = 500  # Lost sales cost (shortage cost)
Twarm = 14  # Number of warmup days

n_obs = len(fcst_2018._data)  # Assign demand vector (from csv) to D_obs: this is observed demand

def S_rmse_alt(d, squared_errors, z):
    rmse = math.sqrt(np.mean(squared_errors))
    return round(d + z*rmse)

I = [0, 0, 0, 0, 0]  # Initialise inventory array of only 0's
TotWaste = 0  # Initialise waste
TotOrdered = 0  # Initialise total ordered
TotDemand = 0 # Initialise total demand
NShort = 0  # Initialise shortage
TC = 0 # Initialise total cost variable
fillrateDay = [] # Initialise fill rate per day array
squared_errors = [] # Initialise empty array for squared errors per day

for t in range(Twarm, n_obs-1):
    mu_t, mu_t1 = fcst_2018.get_prediction_for_day(t) #Returns tuple (t, t+1) pred
    mu_lr = mu_t + mu_t1 # Forecast demand for today and tomorrow
    
#   Calculate squared error based on d_{t} and d_{t+1} and append to list
#   of squared errors
    d_obs_t, d_obs_t1 = fcst_2018.get_observed_demand_for_day(t)
    squared_error = ((d_obs_t + d_obs_t1) - mu_lr)**2
    squared_errors.append(squared_error)

    # Determine s_t based on RMSE-like fcst error
    s_t = S_rmse_alt(mu_lr, squared_errors, z)
    q = max(0, (s_t - sum(I)))  # Determine order quantity q for next day
    d_observed = D_obs[t]  # Read value from array of demand D_obs
    TotDemand += d_observed # Add current demand to total to obtain sum after
    # warm-up period

    # Determine fill rate for the day
    if d_observed == 0:
        fillrateDay.append(1)  # Set fill rate to 1 in case demand is 0

    elif d_observed >= sum(I): # If demand is larger than total inventory
        fillrateDay.append(sum(I) / d_observed) # Append fill rate to fill rate list
        today_short = d_observed - sum(I)# We have shortage: lost sales
        NShort += today_short # Add shortage to total shortage
        I = [0, 0, 0, 0, 0] # Reset I to only zeros, because we have sold out

    else:
        fillrateDay.append(1)
        for i in range(m):  # Inventory is taken full FIFO (old -> new)

            # Procedure for handling demand; waterfall like approach
            if I[i] > d_observed: # If we have more inventory of exp date i than demand
                I[i] = I[i] - d_observed # Remaining inventory is lessened
                d_observed = 0 # D observed is fulfilled, so set to 0 and break out of loop 
                break
            else: # If d_obs >= I[i]; handle demand and continue looping
                d_observed = d_observed - I[i]
                I[i] = 0

    # Determine stats at day's end
    today_waste = I[0]  # Count expired products as waste
    TotWaste += today_waste
    TotOrdered += q  # Add ordered quantity to total ordered quantity
    TC += (u * today_short) + (w * today_waste)
    today_short = 0 # Reset today_short to 0 again as it's not always assigned

    # Shift the day window and add newly ordered goods to inventory
    I = I[1:]
    I.append(q)

result_stats = {
    "Waste": round(TotWaste/TotOrdered, 3),
    "Short": round(NShort/TotDemand, 3),
    "Avg cost (day)": round(TC / (n_obs - Twarm)),
}

print(json.dumps(result_stats, indent=3))